In [ ]:
# Direct Preference Optimization: Your Language Model is Secretly a Reward Model
# https://arxiv.org/abs/2305.18290

In [2]:
import torch
import datasets
import transformers

$$
\mathcal{L}_{\mathrm{DPO}}\left(\pi_\theta ; \pi_{\mathrm{ref}}\right)=-\mathbb{E}_{\left(x, y_w, y_l\right) \sim \mathcal{D}}\left[\log \sigma\left(\beta \log \frac{\pi_\theta\left(y_w \mid x\right)}{\pi_{\mathrm{ref}}\left(y_w \mid x\right)}-\beta \log \frac{\pi_\theta\left(y_l \mid x\right)}{\pi_{\mathrm{ref}}\left(y_l \mid x\right)}\right)\right]
$$

where,
- $\pi_\theta$ policy model (LLM to optimize)
- $ \pi_{\mathrm{ref}}$ reference model (original LLM before optimization)
- $\mathbb{E}$ is expected value
- $\sigma$ logistsic sigmoid function
- $\beta$ hyperparameter to control the divergence between the $\pi_{\theta}$ and $\pi_{ref}$